# Importing libraries and data

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split

In [2]:
import pandas as pd
import numpy as np

In [3]:
# Cufflinks wrapper on plotly
import cufflinks
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from plotly.offline import iplot
cufflinks.go_offline()

# Set global theme

import plotly.figure_factory as ff

import plotly.graph_objects as go

## Importing data

After removing duplicate entries in same quarter we are left with 80 k entries unique in terms of their collection period and offer id. Our features vary from property characteristics such as size, price, number of rooms to location features such as distance to subway, drivetime to center or number of nearby restaurants. 

In [4]:
df = pd.read_csv("price_analysis_data.csv")

In [131]:
df.shape

(86090, 72)

In [132]:
df.columns

Index(['index', 'Id', 'offer_date', 'Area', 'Price', 'latitude', 'longitude',
       'build_year', 'building_floors_num', 'rooms_num', 'lon_mod', 'lat_mod',
       'geoId', 'City', 'subdistrict', 'market', 'address',
       'Building_material', 'Building_ownership', 'Building_type',
       'Construction_status', 'floor_no', 'Heating', 'Windows_type', 'url',
       'Equipment_types_dishwasher', 'Equipment_types_fridge',
       'Equipment_types_furniture', 'Equipment_types_oven',
       'Equipment_types_stove', 'Equipment_types_tv',
       'Equipment_types_washing_machine', 'Extras_types_air_conditioning',
       'Extras_types_attic', 'Extras_types_balcony', 'Extras_types_basement',
       'Extras_types_garage', 'Extras_types_garden', 'Extras_types_lift',
       'Extras_types_separate_kitchen', 'Extras_types_terrace',
       'Extras_types_two_storey', 'Extras_types_usable_room',
       'Media_types_cable-television', 'Media_types_cable_television',
       'Media_types_electricity', 'Medi

In [134]:
df.head()

,index,Id,offer_date,Area,Price,latitude,longitude,build_year,building_floors_num,rooms_num,...,time_transit,restaurant_price_level,restaurant_mean_rating,restaurant_mean_popularity,restaurant_count,restaurant_ratings_count,distance_to_subway,nearest_subway,east_bank,offer_date_q
0,20204,61017653,2020-12-28,198.0,1782000.0,52.286930,21.026510,2001.0,4.0,6,...,25.216667,1.666667,4.340000,201.700000,10,2017,2.339944,Trocka,1,2020Q4
1,832,61234785,2020-12-28,75.0,793500.0,52.224724,21.093448,2008.0,16.0,3,...,35.983333,1.473684,4.355172,238.827586,29,6926,4.231919,Stadion Narodowy,1,2020Q4
2,7117,60981957,2020-12-28,47.6,648900.0,52.197030,20.980030,1970.0,3.0,3,...,34.566667,1.727273,4.218750,237.588235,17,4039,2.199701,Racławicka,0,2020Q4
3,1749,60822327,2020-12-28,68.9,630000.0,52.224940,21.092750,2011.0,8.0,2,...,35.983333,1.473684,4.355172,238.827586,29,6926,4.179323,Stadion Narodowy,1,2020Q4
4,1392,61247094,2020-12-28,26.0,329900.0,52.226714,21.091400,1999.0,5.0,1,...,35.983333,1.473684,4.355172,238.827586,29,6926,3.989925,Stadion Narodowy,1,2020Q4


# EDA - BI style analysis

## Mean and quantiles

In [6]:
df_temp=df[["offer_date_q","unit_price"]].groupby("offer_date_q", as_index=False).mean()
df_temp["top_quantile"]=df[["offer_date_q","unit_price"]].groupby("offer_date_q", as_index=False).quantile(0.8).unit_price
df_temp["bottom_quantile"]=df[["offer_date_q","unit_price"]].groupby("offer_date_q", as_index=False).quantile(0.2).unit_price
df_temp["sample"]=df[["offer_date_q","unit_price"]].groupby("offer_date_q", as_index=False).count().unit_price

In [7]:
#Creating trend variable by dividing each variable by its Q2 value
df_temp["unit_price_static"]=round(df_temp["unit_price"]/df_temp.unit_price[0],4)*100
df_temp["bottom_quantile_static"]=round(df_temp["bottom_quantile"]/df_temp.bottom_quantile[0],4)*100
df_temp["top_quantile_static"]=round(df_temp["top_quantile"]/df_temp.top_quantile[0],4)*100

In [8]:

trace0=go.Scatter(
            x=df_temp.offer_date_q,
            y=df_temp.unit_price,
            mode='lines',
            marker=dict(
            color="blue",
            size=5
            
            ),
        name="mean",

        )


trace1=go.Scatter(
            x=df_temp.offer_date_q,
            y=df_temp.top_quantile,
            mode='lines',
            marker=dict(
            color="green",
            size=5
            
            ),
        name="top quintile",

        )

trace2=go.Scatter(
            x=df_temp.offer_date_q,
            y=df_temp.bottom_quantile,
            mode='lines',
            marker=dict(
            color="red",
            size=5
            
            ),
        name="bottom quintile",

        )

data=[trace0, trace1, trace2]
    
figure=go.Figure(
    data=data,
    layout=go.Layout(
        title="Price per m2 trend [PLN/m2]",
            yaxis=dict(title="Price"),
        xaxis=dict(title="Quarter",type="category")

    
    ))
iplot(figure)



In [9]:

trace0=go.Scatter(
            x=df_temp.offer_date_q,
            y=df_temp.unit_price_static,
            mode='lines',
            marker=dict(
            color="blue",
            size=5
            
            ),
        name="mean",

        )


trace1=go.Scatter(
            x=df_temp.offer_date_q,
            y=df_temp.top_quantile_static,
            mode='lines',
            marker=dict(
            color="green",
            size=5
            
            ),
        name="top quintile",

        )

trace2=go.Scatter(
            x=df_temp.offer_date_q,
            y=df_temp.bottom_quantile_static,
            mode='lines',
            marker=dict(
            color="red",
            size=5
            
            ),
        name="bottom quintile",

        )

data=[trace0, trace1, trace2]
    
figure=go.Figure(
    data=data,
    layout=go.Layout(
        title="Price per m2 trend [2020Q2=100]",
            yaxis=dict(title="Price"),
        xaxis=dict(title="Quarter",type="category")

    
    ))
iplot(figure)




## Sizes

In [10]:
df_temp = df[["offer_date_q","unit_price","Area"]]

In [11]:
# Segmenting data into size buckets
bins = [0,38, 60,90,200]
labels = ["small","medium","large","very large"]
df_temp ['Area_group'] = pd.cut(df.Area,bins= bins, labels = labels, retbins=False)

In [12]:
df_graph=df_temp[["offer_date_q","unit_price",'Area_group']].groupby(["offer_date_q",'Area_group'], as_index=False).mean()

In [13]:
df_graph_Q2 = df_graph.query("offer_date_q == '2020Q2'")
df_graph_Q2.rename(columns = {"unit_price":"q2_unit_price"},inplace=True)
df_graph_Q2.drop(columns = ["offer_date_q"],inplace=True)
df_graph = df_graph.merge(df_graph_Q2, on =["Area_group"])
df_graph["unit_price_static"] = round(df_graph.unit_price / df_graph.q2_unit_price*100,2)

In [14]:
data = []
for area_group in df_graph.Area_group.unique():
    df_temp = df_graph.query("Area_group == '{}'".format(area_group))
    
    
    trace=go.Scatter(
                x=df_temp.offer_date_q,
                y=df_temp.unit_price,
                mode='lines',
                marker=dict(
                size=5

                ),
            name=area_group,

            )
    data.append(trace)
    
figure=go.Figure(
    data=data,
    layout=go.Layout(
        title="Price per m2 segmented by property size",
            yaxis=dict(title="Price"),
        xaxis=dict(title="Month",type="category")

    
    ))
iplot(figure)




In [15]:
data = []
for area_group in df_graph.Area_group.unique():
    df_temp = df_graph.query("Area_group == '{}'".format(area_group))
    
    
    trace=go.Scatter(
                x=df_temp.offer_date_q,
                y=df_temp.unit_price_static,
                mode='lines',
                marker=dict(
                size=5

                ),
            name=area_group,

            )
    data.append(trace)
    
figure=go.Figure(
    data=data,
    layout=go.Layout(
        title="Price per m2 trend [2020Q2=100]",
            yaxis=dict(title="Price"),
        xaxis=dict(title="Quarter",type="category")

    
    ))
iplot(figure)




## Location

In [16]:
df_temp = df[["offer_date_q","unit_price","distance_driving"]]

In [18]:
# Segmenting data into location buckets
bins = [0,3, 5,10,25]
labels = ["strict center","near center","moderate distance","outskirts"]
df_temp ['Area_group'] = pd.cut(df.distance_driving,bins= bins, labels = labels, retbins=False)

In [19]:
df_temp.groupby('Area_group').count()

,offer_date_q,unit_price,distance_driving
Area_group,,,
strict center,8883,8883,8883
near center,16454,16454,16454
moderate distance,37493,37493,37493
outskirts,23260,23260,23260


In [20]:
df_graph=df_temp[["offer_date_q","unit_price",'Area_group']].groupby(["offer_date_q",'Area_group'], as_index=False).mean()

In [21]:
df_graph_Q2 = df_graph.query("offer_date_q == '2020Q2'")
df_graph_Q2.rename(columns = {"unit_price":"q2_unit_price"},inplace=True)
df_graph_Q2.drop(columns = ["offer_date_q"],inplace=True)
df_graph = df_graph.merge(df_graph_Q2, on =["Area_group"])
df_graph["unit_price_static"] = round(df_graph.unit_price / df_graph.q2_unit_price*100,2)

In [22]:
data = []
for area_group in df_graph.Area_group.unique():
    df_temp = df_graph.query("Area_group == '{}'".format(area_group))
    
    
    trace=go.Scatter(
                x=df_temp.offer_date_q,
                y=df_temp.unit_price,
                mode='lines',
                marker=dict(
                size=5

                ),
            name=area_group,

            )
    data.append(trace)
    
figure=go.Figure(
    data=data,
    layout=go.Layout(
        title="Price per m2 segmented by distance to center",
            yaxis=dict(title="Price"),
        xaxis=dict(title="Quarter",type="category")

    
    ))
iplot(figure)




In [23]:
data = []
for area_group in df_graph.Area_group.unique():
    df_temp = df_graph.query("Area_group == '{}'".format(area_group))
    
    
    trace=go.Scatter(
                x=df_temp.offer_date_q,
                y=df_temp.unit_price_static,
                mode='lines',
                marker=dict(
                size=5

                ),
            name=area_group,

            )
    data.append(trace)
    
figure=go.Figure(
    data=data,
    layout=go.Layout(
        title="Price per m2 trend [2020Q2=100]",
            yaxis=dict(title="Price"),
        xaxis=dict(title="Quarter",type="category")

    
    ))
iplot(figure)




# Transforming data for models

## Converting categorical features to one-hot columns

In order to analyze feature importance or use our data in models, we need to transform all categorical features to one-hot columns. 

In [24]:
unique_features=["district","Building_material","Building_type","Construction_status","Heating",
"Windows_type","market","Building_ownership",'offer_date_q']

In [25]:
from sklearn.preprocessing import OneHotEncoder

In [26]:
df_cat=df[unique_features]

In [27]:
enc=OneHotEncoder()
enc.fit(df_cat)

one_hot_val=enc.fit_transform(df_cat).toarray().astype(int)
one_hot_columns=enc.get_feature_names(unique_features)

OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='error', sparse=True)

In [28]:
df_one_hot=pd.DataFrame(one_hot_val, columns=one_hot_columns)

In [29]:
df_one_hot.columns

Index(['district_Bemowo', 'district_Bialoleka', 'district_Bielany',
       'district_Downtown', 'district_Mokotow', 'district_Ochota',
       'district_Other', 'district_Praga', 'district_Southern Praga',
       'district_Subburbs', 'district_Targowek', 'district_Ursynow',
       'district_Wawer', 'district_Wilanow', 'district_Wlochy',
       'district_Wola', 'district_Zoliborz', 'Building_material_breezeblock',
       'Building_material_brick', 'Building_material_cellular_concrete',
       'Building_material_concrete', 'Building_material_concrete_plate',
       'Building_material_hydroton', 'Building_material_not_specified',
       'Building_material_other', 'Building_material_reinforced_concrete',
       'Building_material_silikat', 'Building_material_wood',
       'Building_type_apartment', 'Building_type_block', 'Building_type_house',
       'Building_type_infill', 'Building_type_loft',
       'Building_type_not_specified', 'Building_type_ribbon',
       'Building_type_tenement', '

## Dropping one column from each feature to avoid colinearity as we will be working with linear regression models

First let's drop all "not_specified" columns as they do not carry any information 

In [30]:
not_specified_idx=[]
for i in range(0,df_one_hot.columns.shape[0]):
    if "not_specified" in df_one_hot.columns[i]:
        not_specified_idx.append(i)
not_specified_idx=np.asarray(not_specified_idx)

We also need to drop one column for district, market, nearest_subway and offer_date features. They will serve as a baseline for our linear regression model.

In [31]:
drop_collinear_cols=list(df_one_hot.columns[not_specified_idx])
drop_collinear_cols.append('district_Mokotow')
drop_collinear_cols.append('market_secondary')
drop_collinear_cols.append('nearest_subway_Swietokrzyska')
drop_collinear_cols.append('offer_date_q_2020Q2')

In [32]:
# Substract components of two lists, while keeping order of the remaining components
def list_diff(list1, list2):
    out = []
    for item in list1:
        if not item in list2:
            out.append(item)
    return out

In [33]:
df_cat_columns=list_diff(df_one_hot.columns,drop_collinear_cols)

In [34]:
df_cat=df_one_hot[df_cat_columns]

In [35]:
columns_base=list_diff(df.columns, unique_features)

In [36]:
# Dropping columns not needed in our model
drop_columns_base=[
    'Price','latitude','longitude','lon_mod','lat_mod','grid_price','sample_size','City','subdistrict', 'geoId', 'address', 'floor_no',
     'nearest_subway' ]

In [37]:
columns_base=list_diff(columns_base, drop_columns_base)

In [38]:
df_base=df[columns_base]


In [39]:
df_base.columns

Index(['index', 'Id', 'offer_date', 'Area', 'build_year',
       'building_floors_num', 'rooms_num', 'url', 'Equipment_types_dishwasher',
       'Equipment_types_fridge', 'Equipment_types_furniture',
       'Equipment_types_oven', 'Equipment_types_stove', 'Equipment_types_tv',
       'Equipment_types_washing_machine', 'Extras_types_air_conditioning',
       'Extras_types_attic', 'Extras_types_balcony', 'Extras_types_basement',
       'Extras_types_garage', 'Extras_types_garden', 'Extras_types_lift',
       'Extras_types_separate_kitchen', 'Extras_types_terrace',
       'Extras_types_two_storey', 'Extras_types_usable_room',
       'Media_types_cable-television', 'Media_types_cable_television',
       'Media_types_electricity', 'Media_types_internet', 'Media_types_phone',
       'Media_types_sewage', 'Media_types_water', 'Security_types_alarm',
       'Security_types_anti_burglary_door', 'Security_types_closed_area',
       'Security_types_entryphone', 'Security_types_monitoring',
      

In [40]:
df_base["rooms_num"]=pd.to_numeric(df_base["rooms_num"]) 

## Combining data for modeling

In [41]:
ml_data=pd.concat([df_base,df_cat],axis=1)

In [42]:
ml_data.shape

(86090, 104)

In [43]:
ml_data.head()

,index,Id,offer_date,Area,build_year,building_floors_num,rooms_num,url,Equipment_types_dishwasher,Equipment_types_fridge,...,Windows_type_aluminium,Windows_type_plastic,Windows_type_wooden,market_primary,Building_ownership_co_operative_ownership,Building_ownership_co_operative_ownership_with_a_land_and_mortgage_registe,Building_ownership_full_ownership,Building_ownership_share,offer_date_q_2020Q3,offer_date_q_2020Q4
0,20204,61017653,2020-12-28,198.0,2001.0,4.0,6,https://www.otodom.pl/pl/oferta/200-m2-2-pozio...,0,0,...,0,0,0,0,0,0,1,0,0,1
1,832,61234785,2020-12-28,75.0,2008.0,16.0,3,https://www.otodom.pl/pl/oferta/goclaw-bora-ko...,0,0,...,0,1,0,0,0,0,1,0,0,1
2,7117,60981957,2020-12-28,47.6,1970.0,3.0,3,https://www.otodom.pl/pl/oferta/3-pokoje-ochot...,0,0,...,0,0,0,0,0,0,1,0,0,1
3,1749,60822327,2020-12-28,68.9,2011.0,8.0,2,https://www.otodom.pl/pl/oferta/swietnie-skomu...,0,0,...,0,0,0,0,0,1,0,0,0,1
4,1392,61247094,2020-12-28,26.0,1999.0,5.0,1,https://www.otodom.pl/pl/oferta/kawalerka-na-p...,0,0,...,0,0,1,0,0,1,0,0,0,1


In [44]:
ml_data.columns[:]

Index(['index', 'Id', 'offer_date', 'Area', 'build_year',
       'building_floors_num', 'rooms_num', 'url', 'Equipment_types_dishwasher',
       'Equipment_types_fridge',
       ...
       'Windows_type_aluminium', 'Windows_type_plastic', 'Windows_type_wooden',
       'market_primary', 'Building_ownership_co_operative_ownership',
       'Building_ownership_co_operative_ownership_with_a_land_and_mortgage_registe',
       'Building_ownership_full_ownership', 'Building_ownership_share',
       'offer_date_q_2020Q3', 'offer_date_q_2020Q4'],
      dtype='object', length=104)

# Selecting best features and splitting data

In [141]:
counts, bins = np.histogram(ml_data.unit_price, bins=range(0, 30000, 1000))
bins = bins[1:]
import plotly.express as px
fig = px.bar(x=bins, y=counts, labels={'x': 'unit price PLN/m2', 'y':'count'}, title = 'Unit price distribution')
fig.show()

We can see that our price distribution is slightly skewed, this might make log model more reliable

In [142]:
#Preparing explained variable and model features
X= ml_data.copy()
X=X.query(" unit_price<=30000 and unit_price>=5000")
y=X.unit_price
X.drop(columns=["unit_price","offer_date","Id","url","index"],inplace=True)

In [47]:
X.isna().sum().sort_values()

Area                                     0
Building_material_silikat                0
Building_material_reinforced_concrete    0
Building_material_other                  0
Building_material_hydroton               0
                                        ..
Media_types_water                        0
Media_types_sewage                       0
Media_types_phone                        0
Media_types_electricity                  0
offer_date_q_2020Q4                      0
Length: 99, dtype: int64

## Selecting K Best features

To select best features before working with linear models it is best to use of the shelf function such as KBest. As we are facing a regression problem, f_regression will be the optimal scoring function

In [48]:
from sklearn.feature_selection import SelectKBest, f_regression

In [49]:
bestfeatures = SelectKBest(score_func=f_regression, k="all")

In [50]:
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)

E:\Anaconda\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:299: RuntimeWarning:

invalid value encountered in true_divide

E:\Anaconda\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning:

invalid value encountered in greater

E:\Anaconda\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning:

invalid value encountered in less

E:\Anaconda\lib\site-packages\scipy\stats\_distn_infrastructure.py:1831: RuntimeWarning:

invalid value encountered in less_equal



In [51]:
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']

### Analyzing top features

In [52]:
featureScores.nlargest(50,'Score')

,Specs,Score
36,distance_driving,38536.379416
38,distance_transit,37951.868939
45,distance_to_subway,31867.446802
37,time_driving,31159.453830
39,time_transit,28073.459914
43,restaurant_count,20260.722345
50,district_Downtown,17006.789644
44,restaurant_ratings_count,15122.374279
42,restaurant_mean_popularity,14506.059305
46,east_bank,11903.217308


In [55]:
top_scores = featureScores.nlargest(30,'Score').sort_values(by="Score", ascending=False)

import plotly.express as px
fig = px.bar(top_scores, x='Specs', y='Score')
fig.show()

In [ ]:
## To avoid overfitting we will use top 30 features for our linear models
top_features=featureScores.nlargest(30,'Score').Specs.unique()
top_features

In [56]:
X_top=X[top_features]

In [57]:
## We will use wider set of 50 top features for our DNN models as it is able to learn more complex relations between features
top_features_DNN=featureScores.nlargest(50,'Score').Specs.unique()

## Spliting into train and test sets

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X_top, y, test_size=0.2, random_state=10)

In [62]:
y_train.reset_index(drop=True,inplace=True)
y_test.reset_index(drop=True,inplace=True)

X_train.reset_index(drop=True,inplace=True)
X_test.reset_index(drop=True,inplace=True)

# Statistical approach

## Base model

In [63]:
import statsmodels.api as sm

E:\Anaconda\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.

E:\Anaconda\lib\site-packages\statsmodels\compat\pandas.py:23: FutureWarning:

The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version



In [64]:
X_stats = X_top.copy()
X_stats[['offer_date_q_2020Q4','offer_date_q_2020Q3']] = X[['offer_date_q_2020Q4','offer_date_q_2020Q3']]

In [65]:
X_stats.drop(columns = ["time_driving","distance_transit","district_Bialoleka"],inplace=True)

In [66]:
## Changing build year data as relational to 2000 to make it easier to interpret
X_stats.build_year = X_stats.build_year-2000

In [67]:
# Adding constant - important to make your model work well
X_stats=sm.add_constant(X_stats)

In [68]:
# Selecting WLS model due to price Heteroscedasticity
model = sm.WLS(y,X_stats)
results = model.fit()

In [69]:

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:             unit_price   R-squared:                       0.520
Model:                            WLS   Adj. R-squared:                  0.519
Method:                 Least Squares   F-statistic:                     3210.
Date:                Thu, 07 Jan 2021   Prob (F-statistic):               0.00
Time:                        19:10:31   Log-Likelihood:            -7.6883e+05
No. Observations:               86090   AIC:                         1.538e+06
Df Residuals:                   86060   BIC:                         1.538e+06
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
const                              5639.1540    296.412     19.025      0.000    5058.190    6220.118
distance_driving                   -146.3122      3.531    -41.441      0.000    -153.232    -139.392
distance_to_subway                 -215.8042      6.555    -32.925      0.000    -228.651    -202.957
time_transit                         -6.2527      1.377     -4.542      0.000      -8.951      -3.554
restaurant_count                      9.2051      0.618     14.884      0.000       7.993      10.417
district_Downtown                  1334.0148     31.677     42.113      0.000    1271.929    1396.101
restaurant_ratings_count             -0.0130      0.001    -20.346      0.000      -0.014      -0.012
restaurant_mean_popularity            1.0130      0.061     16.583      0.000       0.893       1.133
east_bank                          -803.4996     20.857    -38.524      0.000    -844.379    -762.620
restaurant_mean_rating             1564.3468     67.909     23.036      0.000    1431.245    1697.448
restaurant_price_level              608.4538     39.309     15.479      0.000     531.408     685.500
Building_type_block                -540.0570     18.209    -29.659      0.000    -575.747    -504.367
Building_type_tenement              914.5052     32.580     28.070      0.000     850.649     978.361
district_Subburbs                  -754.2115     37.257    -20.243      0.000    -827.235    -681.188
Building_type_apartment             687.5303     20.553     33.451      0.000     647.246     727.815
market_primary                     -500.4282     19.763    -25.322      0.000    -539.163    -461.693
Windows_type_wooden                 596.8166     22.349     26.705      0.000     553.014     640.620
Construction_status_ready_to_use    510.8526     15.723     32.490      0.000     480.035     541.670
district_Wola                      -182.8824     22.263     -8.215      0.000    -226.518    -139.247
build_year                           11.9124      0.364     32.736      0.000      11.199      12.626
rooms_num                          -385.1830      6.486    -59.391      0.000    -397.895    -372.471
district_Targowek                  -664.8891     37.051    -17.945      0.000    -737.509    -592.269
Extras_types_balcony                -98.3972     13.680     -7.193      0.000    -125.210     -71.585
Building_ownership_full_ownership   170.9295     13.653     12.519      0.000     144.170     197.689
Extras_types_air_conditioning       406.7737     28.533     14.256      0.000     350.849     462.698
floor_num                            37.3479      2.386     15.652      0.000      32.671      42.025
Building_material_brick            -121.9764     16.430     -7.424      0.000    -154.179     -89.774
Construction_status_to_

In [71]:
# Checking if selected features are statistically significant 
alpha=0.05
results.pvalues<alpha

const                                True
distance_driving                     True
distance_to_subway                   True
time_transit                         True
restaurant_count                     True
district_Downtown                    True
restaurant_ratings_count             True
restaurant_mean_popularity           True
east_bank                            True
restaurant_mean_rating               True
restaurant_price_level               True
Building_type_block                  True
Building_type_tenement               True
district_Subburbs                    True
Building_type_apartment              True
market_primary                       True
Windows_type_wooden                  True
Construction_status_ready_to_use     True
district_Wola                        True
build_year                           True
rooms_num                            True
district_Targowek                    True
Extras_types_balcony                 True
Building_ownership_full_ownership 

## Log model

In [72]:
y_log=np.log(y)

In [73]:
X_stats_log = X_top.copy()
X_stats_log[['offer_date_q_2020Q4','offer_date_q_2020Q3']] = X[['offer_date_q_2020Q4','offer_date_q_2020Q3']]
X_stats_log=sm.add_constant(X_stats)

In [74]:
X_stats_log.drop(columns = ["time_transit"],inplace=True)

In [75]:
# Selecting WLS model due to price Heteroscedasticity
model = sm.WLS(y_log,X_stats_log)
results = model.fit()

In [76]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:             unit_price   R-squared:                       0.553
Model:                            WLS   Adj. R-squared:                  0.553
Method:                 Least Squares   F-statistic:                     3798.
Date:                Thu, 07 Jan 2021   Prob (F-statistic):               0.00
Time:                        19:10:33   Log-Likelihood:                 40414.
No. Observations:               86090   AIC:                        -8.077e+04
Df Residuals:                   86061   BIC:                        -8.050e+04
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
const                                 8.7733      0.025    357.839      0.000       8.725       8.821
distance_driving                     -0.0150      0.000    -61.763      0.000      -0.015      -0.014
distance_to_subway                   -0.0220      0.001    -43.321      0.000      -0.023      -0.021
restaurant_count                      0.0009    5.1e-05     17.386      0.000       0.001       0.001
district_Downtown                     0.0922      0.003     35.221      0.000       0.087       0.097
restaurant_ratings_count          -1.298e-06    5.3e-08    -24.503      0.000    -1.4e-06   -1.19e-06
restaurant_mean_popularity         8.347e-05   5.01e-06     16.670      0.000    7.37e-05    9.33e-05
east_bank                            -0.0797      0.002    -47.234      0.000      -0.083      -0.076
restaurant_mean_rating                0.1580      0.006     28.146      0.000       0.147       0.169
restaurant_price_level                0.0393      0.003     12.298      0.000       0.033       0.046
Building_type_block                  -0.0451      0.002    -29.917      0.000      -0.048      -0.042
Building_type_tenement                0.0705      0.003     26.159      0.000       0.065       0.076
district_Subburbs                    -0.0785      0.003    -25.710      0.000      -0.084      -0.072
Building_type_apartment               0.0568      0.002     33.415      0.000       0.053       0.060
market_primary                       -0.0420      0.002    -25.693      0.000      -0.045      -0.039
Windows_type_wooden                   0.0447      0.002     24.166      0.000       0.041       0.048
Construction_status_ready_to_use      0.0466      0.001     35.802      0.000       0.044       0.049
district_Wola                        -0.0229      0.002    -12.504      0.000      -0.026      -0.019
build_year                            0.0010   3.01e-05     33.647      0.000       0.001       0.001
rooms_num                            -0.0384      0.001    -71.627      0.000      -0.039      -0.037
district_Targowek                    -0.0585      0.003    -19.092      0.000      -0.064      -0.052
Extras_types_balcony                 -0.0091      0.001     -8.028      0.000      -0.011      -0.007
Building_ownership_full_ownership     0.0160      0.001     14.210      0.000       0.014       0.018
Extras_types_air_conditioning         0.0318      0.002     13.454      0.000       0.027       0.036
floor_num                             0.0030      0.000     15.416      0.000       0.003       0.003
Building_material_brick              -0.0121      0.001     -8.908      0.000      -0.015      -0.009
Construction_status_to_completion     0.0090      0.002      5.675      0.000       0.006       0.012
offer_date_q_2020Q4    

In [77]:
# Checking if selected features are statistically significant 
alpha=0.05
results.pvalues<alpha

const                                True
distance_driving                     True
distance_to_subway                   True
restaurant_count                     True
district_Downtown                    True
restaurant_ratings_count             True
restaurant_mean_popularity           True
east_bank                            True
restaurant_mean_rating               True
restaurant_price_level               True
Building_type_block                  True
Building_type_tenement               True
district_Subburbs                    True
Building_type_apartment              True
market_primary                       True
Windows_type_wooden                  True
Construction_status_ready_to_use     True
district_Wola                        True
build_year                           True
rooms_num                            True
district_Targowek                    True
Extras_types_balcony                 True
Building_ownership_full_ownership    True
Extras_types_air_conditioning     

# Neural networks model

## Spliting into train and test sets

In [78]:
X_dnn = X.query("offer_date_q_2020Q3==0 and offer_date_q_2020Q4==0")[top_features_DNN]

In [79]:
y_dnn = y[X.query("offer_date_q_2020Q3==0 and offer_date_q_2020Q4==0").index]

In [80]:
y_dnn

66991    14717.045455
66992    14290.910981
66993    10416.666667
66994     7000.000000
66995    12079.183376
             ...     
86085    14302.325581
86086    14302.325581
86087    14487.632509
86088    14890.909091
86089    12973.684211
Name: unit_price, Length: 19099, dtype: float64

In [81]:
X_dnn.reset_index(inplace=True,drop=True)
y_dnn.reset_index(drop=True, inplace=True)

In [82]:
X_train, X_test, y_train, y_test = train_test_split(X_dnn, y_dnn, test_size=0.2, random_state=10)

In [83]:
y_train.reset_index(drop=True,inplace=True)
y_test.reset_index(drop=True,inplace=True)

X_train.reset_index(drop=True,inplace=True)
X_test.reset_index(drop=True,inplace=True)

## Transforming and scaling data for NN

In [84]:
def DNN_plot_loss(history, starting_epoch):

        trace0=go.Scatter(
                y=history.history['loss'][starting_epoch:],
                x=history.epoch[starting_epoch:],
                mode='lines',
                marker=dict(
                color="blue",
                size=5,
                opacity=0.5
                ),
                name="Training Loss"
            )


        trace1=go.Scatter(
                y=history.history['val_loss'][starting_epoch:],
                x=history.epoch[starting_epoch:],
                mode='lines',
                marker=dict(
                color="red",
                size=5,
                opacity=0.5
                ),
                name="Validation Loss"
            )

        data=[trace0, trace1]
        figure=go.Figure(
            data=data,
            layout=go.Layout(
                title="Learning curve",
                yaxis=dict(title="Loss"),
                xaxis=dict(title="Epoch",range=(starting_epoch,history.epoch[-1])),
                legend=dict(
                    x=1,
                    y=1,
                    traceorder="normal",
                    font=dict(
                        family="sans-serif",
                        size=12,
                        color="black"
                    ),
                bgcolor=None


            )))
        iplot(figure)

In [85]:
y_train=np.asarray(y_train).reshape(-1,1)
y_train.shape

(15279, 1)

In [86]:
y_test=np.asarray(y_test).reshape(-1,1)
y_test.shape

(3820, 1)

In [87]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [88]:
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [89]:
X_train.shape

(15279, 50)

In [90]:
X_test.shape

(3820, 50)

## DNN Model

In [91]:
def DNN_plot_loss(history, starting_epoch,previous_val_loss):

        trace0=go.Scatter(
                y=history.history['loss'][starting_epoch:],
                x=history.epoch[starting_epoch:],
                mode='lines',
                marker=dict(
                color="blue",
                size=5,
                opacity=0.5
                ),
                name="Training Loss"
            )


        trace1=go.Scatter(
                y=history.history['val_loss'][starting_epoch:],
                x=history.epoch[starting_epoch:],
                mode='lines',
                marker=dict(
                color="red",
                size=5,
                opacity=0.5
                ),
                name="Validation Loss"
            )
        
        trace2=go.Scatter(
                y=list(np.ones([len(history.epoch[starting_epoch:])])*np.asarray(previous_val_loss).min()),
                x=history.epoch[starting_epoch:],
                mode='lines',
                marker=dict(
                color="grey",
                size=5,

                ),
                name="Lowest error from previous models"
            )

        data=[trace0, trace1,trace2]
        figure=go.Figure(
            data=data,
            layout=go.Layout(
                title="Learning curve",
                yaxis=dict(title="Loss",range=(900,1300)),
                xaxis=dict(title="Epoch",range=(starting_epoch,history.epoch[-1])),
                legend=dict(
                    x=0.57,
                    y=1,
                    traceorder="normal",
                    font=dict(
                        family="sans-serif",
                        size=12,
                        color="black"
                    ),
                bgcolor=None,



            )))
        iplot(figure)

In [92]:
def performance_summary(model, X_test, y_test ):
    
    y_hat=model.predict(X_test)
    
    df_summary=pd.DataFrame(y_hat, columns=["y_hat"])
    df_summary["y_true"]=y_test
    df_summary["abs_error"]=np.abs(df_summary.y_true-df_summary.y_hat)
    df_summary["error"]=df_summary.y_hat-df_summary.y_true
    df_summary["relative_error"]= df_summary["error"]/df_summary.y_true
    df_summary["relative_abs_error"]= df_summary["abs_error"]/df_summary.y_true
    df_summary["diff_to_benchmark"]= df_summary.y_true / df_summary.y_hat -1
    df_summary["variation_from_benchmark"]= df_summary.y_true  - df_summary.y_hat 
    
    share_within_5pct=(df_summary.query("relative_abs_error<0.05").shape[0]/df_summary.shape[0])*100
    
    print("{:.2f}% : Share of forecasts within 5% absolute error\n".format(share_within_5pct))
    print("{:.2f}   : Mean absolute error \n".format(df_summary.abs_error.mean()))
    print("{:.2f}% : Mean absolute percentage error\n".format(df_summary.relative_abs_error.mean()*100))
    
    return(df_summary)



In [93]:
tf.keras.backend.clear_session()
tf.random.set_seed(60)

model=keras.models.Sequential([
    
    keras.layers.Dense(1024, input_dim = X_train.shape[1]), 
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.5),
    
    keras.layers.Dense(512),  
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),

    keras.layers.Dense(512),  
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    
    keras.layers.Dense(units=256), 
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.2),
    
    keras.layers.Dense(units=256), 
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.02),

    
    keras.layers.Dense(units=128),
    keras.layers.LeakyReLU(), 
    keras.layers.Dropout(0.01),
    keras.layers.Dense(units=1, activation="linear"),



],name="Learning_rate_decay",)


In [94]:
model.summary()

Model: "Learning_rate_decay"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1024)              52224     
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 1024)              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 1024)              4096      
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)       

In [95]:
# Calculating learning rate decay
learning_rate = 0.005
decay = 5e-4
n_epochs=400
n_steps_per_epoch = len(X_train) // 1024
epochs = np.arange(n_epochs)
lrs = learning_rate / (1 + decay * epochs * n_steps_per_epoch)

In [96]:
# Visualizing learning rate decay

trace0=go.Scatter(
            y=lrs,
            x=epochs,
            mode='lines',
            marker=dict(
            color="red",
            size=5,
            opacity=0.5
            )
    )
        

data=[trace0]
figure=go.Figure(
            data=data,
            layout=go.Layout(
                title="Learning rate decay",
                yaxis=dict(title="Learning rate"),
                xaxis=dict(title="Epoch"),
                legend=dict(
                    x=1,
                    y=1,
                    traceorder="normal",
                    font=dict(
                        family="sans-serif",
                        size=12,
                        color="black"
                    ),
                bgcolor=None


            )))
iplot(figure)

In [97]:
%%time
#Added learning rate decay to Adam optimizer
optimizer = keras.optimizers.Adam(lr=0.005, decay=5e-4)



model.compile(optimizer=optimizer, warm_start=False, 
            loss='mean_absolute_error')


history = model.fit(X_train, y_train,
                    epochs=200, batch_size=1024,
                    validation_data=(X_test, y_test), 
                    verbose=1)

Train on 15279 samples, validate on 3820 samples
Epoch 1/200
15279/15279 [==============================] - 17s 1ms/sample - loss: 11138.6754 - val_loss: 10239.3035
Epoch 2/200
15279/15279 [==============================] - 0s 31us/sample - loss: 11033.1775 - val_loss: 9840.1544
Epoch 3/200
15279/15279 [==============================] - 0s 18us/sample - loss: 10753.4937 - val_loss: 8967.3857
Epoch 4/200
15279/15279 [==============================] - 0s 22us/sample - loss: 10177.3446 - val_loss: 8148.3851
Epoch 5/200
15279/15279 [==============================] - 0s 19us/sample - loss: 9140.2583 - val_loss: 4520.9703
Epoch 6/200
15279/15279 [==============================] - 0s 20us/sample - loss: 7454.6206 - val_loss: 4352.6511
Epoch 7/200
15279/15279 [==============================] - 0s 19us/sample - loss: 4969.6093 - val_loss: 3638.5183
Epoch 8/200
15279/15279 [==============================] - 0s 19us/sample - loss: 2289.8628 - val_loss: 10268.6003
Epoch 9/200
15279/15279 [========

Epoch 72/200
15279/15279 [==============================] - 0s 20us/sample - loss: 1058.8535 - val_loss: 1039.0534
Epoch 73/200
15279/15279 [==============================] - 0s 18us/sample - loss: 1061.1445 - val_loss: 1054.8905
Epoch 74/200
15279/15279 [==============================] - 0s 19us/sample - loss: 1066.9739 - val_loss: 1042.1754
Epoch 75/200
15279/15279 [==============================] - 0s 18us/sample - loss: 1058.3064 - val_loss: 1067.1444
Epoch 76/200
15279/15279 [==============================] - 0s 19us/sample - loss: 1052.2062 - val_loss: 1047.2824
Epoch 77/200
15279/15279 [==============================] - 0s 18us/sample - loss: 1059.2324 - val_loss: 1039.3568
Epoch 78/200
15279/15279 [==============================] - 0s 19us/sample - loss: 1052.2024 - val_loss: 1038.2556
Epoch 79/200
15279/15279 [==============================] - 0s 18us/sample - loss: 1056.3721 - val_loss: 1050.8481
Epoch 80/200
15279/15279 [==============================] - 0s 17us/sample - los

Epoch 143/200
15279/15279 [==============================] - 0s 16us/sample - loss: 994.5656 - val_loss: 1011.0154
Epoch 144/200
15279/15279 [==============================] - 0s 16us/sample - loss: 992.3047 - val_loss: 1009.6348
Epoch 145/200
15279/15279 [==============================] - 0s 16us/sample - loss: 988.5884 - val_loss: 1023.9263
Epoch 146/200
15279/15279 [==============================] - 0s 16us/sample - loss: 995.9641 - val_loss: 1020.6820
Epoch 147/200
15279/15279 [==============================] - 0s 16us/sample - loss: 987.4623 - val_loss: 1017.1495
Epoch 148/200
15279/15279 [==============================] - 0s 16us/sample - loss: 991.9338 - val_loss: 1016.9299
Epoch 149/200
15279/15279 [==============================] - 0s 16us/sample - loss: 987.9852 - val_loss: 1012.6893
Epoch 150/200
15279/15279 [==============================] - 0s 16us/sample - loss: 987.0115 - val_loss: 1010.0241
Epoch 151/200
15279/15279 [==============================] - 0s 16us/sample - lo

In [98]:
DNN_plot_loss(history,starting_epoch=10, previous_val_loss=10)

In [99]:
df_summary = performance_summary(model, X_test, y_test )

41.96% : Share of forecasts within 5% absolute error

1003.87   : Mean absolute error 

8.83% : Mean absolute percentage error



In [100]:
df_summary.describe()

,y_hat,y_true,abs_error,error,relative_error,relative_abs_error,diff_to_benchmark,variation_from_benchmark
count,3820.000000,3820.000000,3820.000000,3820.000000,3820.000000,3820.000000,3820.000000,3820.000000
mean,10953.040039,11108.809011,1003.873385,-155.788545,0.001531,0.088296,0.013528,155.788545
std,2136.194336,2625.900090,1050.545174,1444.781780,0.126449,0.090518,0.124504,1444.781780
min,6233.355957,5083.056478,0.232871,-7770.944300,-0.428053,0.000028,-0.603941,-8810.380425
25%,9369.280029,9181.536990,295.199981,-763.355399,-0.066515,0.028563,-0.056795,-588.773847
50%,10811.808594,10863.034483,674.761598,-48.208018,-0.004671,0.063038,0.004693,48.208018
75%,12247.240967,12534.714388,1357.405437,588.773847,0.060215,0.120160,0.071254,763.355399
max,19913.207031,20000.000000,8810.380425,8810.380425,1.524874,1.524874,0.748415,7770.944300


In [101]:
df_summary

,y_hat,y_true,abs_error,error,relative_error,relative_abs_error,diff_to_benchmark,variation_from_benchmark
0,12200.599609,11414.285714,786.313895,786.313895,0.068889,0.068889,-0.064449,-786.313895
1,8024.263672,8210.937500,186.673828,-186.673828,-0.022735,0.022735,0.023264,186.673828
2,11315.282227,11978.465680,663.183453,-663.183453,-0.055365,0.055365,0.058610,663.183453
3,12225.708984,10592.631414,1633.077570,1633.077570,0.154171,0.154171,-0.133577,-1633.077570
4,7843.622070,7992.916175,149.294104,-149.294104,-0.018678,0.018678,0.019034,149.294104
...,...,...,...,...,...,...,...,...
3815,14070.324219,13700.000000,370.324219,370.324219,0.027031,0.027031,-0.026320,-370.324219
3816,8176.102539,7600.000000,576.102539,576.102539,0.075803,0.075803,-0.070462,-576.102539
3817,10280.505859,10599.651655,319.145795,-319.145795,-0.030109,0.030109,0.031044,319.145795
3818,9235.763672,9169.550173,66.213499,66.213499,0.007221,0.007221,-0.007169,-66.213499


In [102]:

trace0=go.Scatter(
            x=df_summary.y_hat,
            y=df_summary.y_true,
            mode='markers',
            marker=dict(
            color="blue",
            size=5,
            opacity=0.25
            
            ),

        )


trace1=go.Scatter(
            x=df_summary.y_true,
            y=df_summary.y_true,
            mode='lines',
            marker=dict(
            color="blue",
            size=5,
            opacity=0.25
            
            ),

        )




        

data=[trace0,trace1]
    
figure=go.Figure(
    data=data,
    layout=go.Layout(
        title="Price per m2 trend [May=100]",
        yaxis=dict(title="Predicted price"),
        xaxis=dict(title="Actual price")

    
    ))
iplot(figure)




In [103]:
df_summary

,y_hat,y_true,abs_error,error,relative_error,relative_abs_error,diff_to_benchmark,variation_from_benchmark
0,12200.599609,11414.285714,786.313895,786.313895,0.068889,0.068889,-0.064449,-786.313895
1,8024.263672,8210.937500,186.673828,-186.673828,-0.022735,0.022735,0.023264,186.673828
2,11315.282227,11978.465680,663.183453,-663.183453,-0.055365,0.055365,0.058610,663.183453
3,12225.708984,10592.631414,1633.077570,1633.077570,0.154171,0.154171,-0.133577,-1633.077570
4,7843.622070,7992.916175,149.294104,-149.294104,-0.018678,0.018678,0.019034,149.294104
...,...,...,...,...,...,...,...,...
3815,14070.324219,13700.000000,370.324219,370.324219,0.027031,0.027031,-0.026320,-370.324219
3816,8176.102539,7600.000000,576.102539,576.102539,0.075803,0.075803,-0.070462,-576.102539
3817,10280.505859,10599.651655,319.145795,-319.145795,-0.030109,0.030109,0.031044,319.145795
3818,9235.763672,9169.550173,66.213499,66.213499,0.007221,0.007221,-0.007169,-66.213499


In [104]:
counts, bins = np.histogram(df_summary.diff_to_benchmark*100-2.5, bins=range(-100, 100, 5))
bins = bins[1:]
import plotly.express as px
fig = px.bar(x=bins, y=counts, labels={'x': 'diff to benchmark', 'y':'count'})
fig.show()

In [105]:
ypred=model.predict(X_test)

# Benchmarking for current offers batch

In [106]:
X_current = ml_data.query("offer_date_q_2020Q4==1")
X_current.reset_index(inplace=True, drop=True)

In [107]:
y_b=X_current.unit_price
X_b = X_current[top_features_DNN]

In [108]:
X_b=scaler.transform(X_b)

In [109]:
y_benchmark = model.predict(X_b)

In [110]:
y_benchmark 

array([[ 8172.88 ],
       [10203.974],
       [10100.407],
       ...,
       [12300.622],
       [11586.406],
       [11350.052]], dtype=float32)

In [111]:
df_benchmark = X_current[["Id","Area","unit_price","url"]]

In [112]:
df_benchmark["unit_price_hat"] = model.predict(X_b)

In [113]:
df_benchmark = df_benchmark.query("unit_price<=30000")

In [114]:
df_benchmark["price"] = df_benchmark.unit_price * df_benchmark.Area
df_benchmark["price_benchmark"] = df_benchmark.unit_price_hat * df_benchmark.Area

In [115]:
df_benchmark["diff_to_benchmark"] = (df_benchmark.unit_price / df_benchmark.unit_price_hat -1)*100

In [116]:
df_benchmark.describe()

,Id,Area,unit_price,unit_price_hat,price,price_benchmark,diff_to_benchmark
count,3.564700e+04,35647.000000,35647.000000,35647.000000,3.564700e+04,3.564700e+04,35647.000000
mean,6.090419e+07,59.535511,11525.259537,11121.745117,6.857077e+05,6.625416e+05,3.802182
std,1.114287e+06,27.208517,2606.509445,2084.127441,3.872745e+05,3.535362e+05,14.052211
min,2.326098e+07,20.000000,5021.676301,5997.999023,1.950000e+05,1.964494e+05,-66.916247
25%,6.093047e+07,41.600000,9674.881085,9569.901367,4.550000e+05,4.422394e+05,-4.698445
50%,6.109075e+07,53.700000,11189.655172,11014.955078,5.783440e+05,5.633730e+05,2.473763
75%,6.120821e+07,68.615000,12937.931034,12391.106445,7.700000e+05,7.571168e+05,10.988874
max,6.133235e+07,200.000000,20000.000000,19799.261719,3.990000e+06,3.523973e+06,132.446424


In [117]:
df_benchmark.head()

,Id,Area,unit_price,url,unit_price_hat,price,price_benchmark,diff_to_benchmark
0,61017653,198.0,9000.000000,https://www.otodom.pl/pl/oferta/200-m2-2-pozio...,8172.879883,1782000.0,1.618230e+06,10.120302
1,61234785,75.0,10580.000000,https://www.otodom.pl/pl/oferta/goclaw-bora-ko...,10203.973633,793500.0,7.652980e+05,3.685097
2,60981957,47.6,13632.352941,https://www.otodom.pl/pl/oferta/3-pokoje-ochot...,10100.407227,648900.0,4.807794e+05,34.968350
3,60822327,68.9,9143.686502,https://www.otodom.pl/pl/oferta/swietnie-skomu...,10428.519531,630000.0,7.185250e+05,-12.320378
4,61247094,26.0,12688.461538,https://www.otodom.pl/pl/oferta/kawalerka-na-p...,11303.499023,329900.0,2.938910e+05,12.252511


In [118]:
counts, bins = np.histogram(df_benchmark.diff_to_benchmark - 2.5, bins=range(-100, 100, 5))
bins = bins[1:]
import plotly.express as px
fig = px.bar(x=bins, y=counts, labels={'x': 'diff to benchmark', 'y':'count'})
fig.show()

In [119]:
df_benchmark.sort_values(by="diff_to_benchmark", ascending=False)

,Id,Area,unit_price,url,unit_price_hat,price,price_benchmark,diff_to_benchmark
10592,61221577,45.00,18444.444444,https://www.otodom.pl/pl/oferta/piekne-mieszka...,7934.922852,830000.0,3.570715e+05,132.446424
5401,61131693,90.59,18974.964124,https://www.otodom.pl/pl/oferta/apartament-ul-...,8209.030273,1718942.0,7.436561e+05,131.147450
16636,58321764,122.74,20000.000000,https://www.otodom.pl/pl/oferta/apartament-4-p...,9283.764648,2454800.0,1.139489e+06,115.429847
3228,59910643,164.00,14939.024390,https://www.otodom.pl/pl/oferta/6-pok-160-m-og...,6991.835938,2450000.0,1.146661e+06,113.663829
4446,61254740,90.56,18981.250000,https://www.otodom.pl/pl/oferta/mieszkanie-w-p...,9110.213867,1718942.0,8.250210e+05,108.351311
...,...,...,...,...,...,...,...,...
35007,61046235,112.00,5348.214286,not_specified,13346.070312,599000.0,1.494760e+06,-59.926674
7816,61332176,80.00,5487.500000,https://www.otodom.pl/pl/oferta/unikatowy-apar...,14131.974609,439000.0,1.130558e+06,-61.169616
30087,60902429,184.50,5414.634146,not_specified,14271.791992,999000.0,2.633146e+06,-62.060587
18465,60753538,80.00,5487.500000,https://www.otodom.pl/oferta/unikatowy-apartam...,14622.193359,439000.0,1.169775e+06,-62.471430


In [120]:
#df_benchmark.query("monthly_price>2500 and diff_to_benchmark >20 and Area < 60").to_excel("price_benchmarking.xlsx")

In [121]:
(df_benchmark["diff_to_benchmark"]*df_benchmark.unit_price/100).max()

24885.18153466175

In [122]:
(df_benchmark["diff_to_benchmark"]*df_benchmark.unit_price/100).mean()

644.7957622510061

In [123]:
(df_benchmark["diff_to_benchmark"]*df_benchmark.unit_price/100).max()
(df_benchmark["diff_to_benchmark"]*df_benchmark.unit_price/100).mean()

24885.18153466175

644.7957622510061

In [124]:
df_benchmark.describe()

,Id,Area,unit_price,unit_price_hat,price,price_benchmark,diff_to_benchmark
count,3.564700e+04,35647.000000,35647.000000,35647.000000,3.564700e+04,3.564700e+04,35647.000000
mean,6.090419e+07,59.535511,11525.259537,11121.745117,6.857077e+05,6.625416e+05,3.802182
std,1.114287e+06,27.208517,2606.509445,2084.127441,3.872745e+05,3.535362e+05,14.052211
min,2.326098e+07,20.000000,5021.676301,5997.999023,1.950000e+05,1.964494e+05,-66.916247
25%,6.093047e+07,41.600000,9674.881085,9569.901367,4.550000e+05,4.422394e+05,-4.698445
50%,6.109075e+07,53.700000,11189.655172,11014.955078,5.783440e+05,5.633730e+05,2.473763
75%,6.120821e+07,68.615000,12937.931034,12391.106445,7.700000e+05,7.571168e+05,10.988874
max,6.133235e+07,200.000000,20000.000000,19799.261719,3.990000e+06,3.523973e+06,132.446424


In [125]:
df_summary_q3 = performance_summary(model, X_b, y_b)

35.34% : Share of forecasts within 5% absolute error

1195.41   : Mean absolute error 

9.94% : Mean absolute percentage error



In [126]:
df_summary = performance_summary(model, X_test, y_test)

41.96% : Share of forecasts within 5% absolute error

1003.87   : Mean absolute error 

8.83% : Mean absolute percentage error



In [127]:
df_summary_q3.describe()

,y_hat,y_true,abs_error,error,relative_error,relative_abs_error,diff_to_benchmark,variation_from_benchmark
count,35647.000000,35647.000000,35647.000000,35647.000000,35647.000000,3.564700e+04,35647.000000,35647.000000
mean,11121.745117,11525.259537,1195.410729,-403.536389,-0.019258,9.935251e-02,0.038022,403.536389
std,2084.127441,2606.509445,1184.803436,1634.001192,0.133369,9.103259e-02,0.140522,1634.001192
min,5997.999023,5021.676301,0.008079,-10765.933851,-0.569793,8.999209e-07,-0.669162,-10922.210938
25%,9569.901367,9674.881085,364.796587,-1189.529446,-0.099009,3.467128e-02,-0.046984,-505.424851
50%,11014.955078,11189.655172,833.721680,-257.367955,-0.024140,7.631358e-02,0.024738,257.367955
75%,12391.106445,12937.931034,1632.958135,505.424851,0.049301,1.388313e-01,0.109889,1189.529446
max,19799.261719,20000.000000,10922.210938,10922.210938,2.022632,2.022632e+00,1.324464,10765.933851


In [128]:
df_summary.describe()

,y_hat,y_true,abs_error,error,relative_error,relative_abs_error,diff_to_benchmark,variation_from_benchmark
count,3820.000000,3820.000000,3820.000000,3820.000000,3820.000000,3820.000000,3820.000000,3820.000000
mean,10953.040039,11108.809011,1003.873385,-155.788545,0.001531,0.088296,0.013528,155.788545
std,2136.194336,2625.900090,1050.545174,1444.781780,0.126449,0.090518,0.124504,1444.781780
min,6233.355957,5083.056478,0.232871,-7770.944300,-0.428053,0.000028,-0.603941,-8810.380425
25%,9369.280029,9181.536990,295.199981,-763.355399,-0.066515,0.028563,-0.056795,-588.773847
50%,10811.808594,10863.034483,674.761598,-48.208018,-0.004671,0.063038,0.004693,48.208018
75%,12247.240967,12534.714388,1357.405437,588.773847,0.060215,0.120160,0.071254,763.355399
max,19913.207031,20000.000000,8810.380425,8810.380425,1.524874,1.524874,0.748415,7770.944300


## Variation to benchmark distribution

In [129]:
trace0 = go.Histogram(
    x = df_summary.diff_to_benchmark, 
    histnorm = 'percent',
    xbins=dict(size = 0.05, start = -0.525, end=0.5),
    marker = dict(
        color="blue",
        opacity=0.5
        ),
        name = "2020Q2"
        
                     )

trace1 = go.Histogram(
    x = df_summary_q3.diff_to_benchmark, 
    histnorm = 'percent',
    xbins=dict(size = 0.05, start = -0.525, end=0.5),
    marker = dict(
        color="red",
        opacity=0.5
        ),
        name = "2020Q4"
        
                     )

data = [trace0,trace1]

figure = go.Figure(
    data = data,
    layout = go.Layout(
    barmode = "overlay",
        
    title = "Price distribution")
        
)

iplot(figure)